In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
import torch

In [3]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
# Load the IMDB dataset
max_features = 10000  # Number of words to consider as features
max_len = 500  # Cut texts after this number of words
batch_size = 64
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Preprocess the data
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [6]:
# Move data to PyTorch tensors and to GPU if available
x_train_tensor = torch.tensor(x_train, dtype=torch.long).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float).to(device)

In [7]:
# Build the RNN model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [8]:
# Move the model to GPU if available
model.to(device)

<Sequential name=sequential, built=False>

In [9]:
# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 205s 655ms/step - accuracy: 0.6950 - loss: 0.5790 - val_accuracy: 0.8754 - val_loss: 0.3257
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 203s 650ms/step - accuracy: 0.9028 - loss: 0.2580 - val_accuracy: 0.8760 - val_loss: 0.3177
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 216s 689ms/step - accuracy: 0.9365 - loss: 0.1804 - val_accuracy: 0.8819 - val_loss: 0.3000
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 206s 658ms/step - accuracy: 0.9585 - loss: 0.1279 - val_accuracy: 0.8821 - val_loss: 0.3341
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 202s 646ms/step - accuracy: 0.9691 - loss: 0.0954 - val_accuracy: 0.8778 - val_loss: 0.3976
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 201s 642ms/step - accuracy: 0.9724 - loss: 0.0842 - val_accuracy: 0.8778 - val_loss: 0.4101
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 202s 645ms/step - accuracy: 0.9781 - loss: 0.0672 - val_accuracy: 0.8701 - val_loss: 0.5099
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 200s 639ms/step - accuracy: 0.9834 -

In [10]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 224s 286ms/step - accuracy: 0.8524 - loss: 0.5758
Test loss: 0.5744
Test accuracy: 0.8542


In [11]:
# Select a random review from the test set
word_index = imdb.get_word_index()
reverse_word_index = {index: word for word, index in word_index.items()}

random_index = np.random.randint(0, len(x_test))
random_review = ' '.join([reverse_word_index.get(i, '?') for i in x_test[random_index] if i in reverse_word_index])


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
def predict_sentiment(review_text, model, maxlen):
    # Tokenize and pad the input sequence
    review_sequence = sequence.pad_sequences([imdb.get_word_index()[word] if word in imdb.get_word_index() else 0 for word in review_text.split()], maxlen=maxlen)

    # Make prediction
    prediction = model.predict(review_sequence)[0, 0]

    sentiment = "positive" if prediction > 0.5 else "negative"
    return sentiment

In [ ]:
print(f"\nRandom Review: {random_review}")
predicted_sentiment = predict_sentiment(random_review, model, max_len)
print(f"Predicted sentiment: {predicted_sentiment}")